##載入資料集、HappyML

In [ ]:
import os

if not os.path.isdir("HappyML"):
  os.system("git clone https://github.com/cnchi/HappyML.git")

In [ ]:

import pandas as pd
import os
if not os.path.isfile("HappyML"):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" )

Dataset_File = "Glass.csv"
if not os.path.isfile(Dataset_File):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" +Dataset_File)

dataset = pd.read_csv("Glass.csv")
dataset

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


##資料前處理

###自、應變數

In [ ]:
import HappyML.preprocessor as pp

In [ ]:
X, Y = pp.decomposition(dataset, x_columns=[i for i in range(0, 9)], y_columns=[9])

In [ ]:
X

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0
...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0


In [ ]:
Y

,Type
0,1
1,1
2,1
3,1
4,1
...,...
209,7
210,7
211,7
212,7


In [ ]:
dataset['Type'].unique()

array([1, 2, 3, 5, 6, 7])

###訓練、測試集

In [ ]:
#X_train, X_test, Y_train, Y_test = pp.split_train_test(x_ary=X, y_ary=Y)
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)

(171, 9)


###類別資料數位化

In [ ]:
from tensorflow.keras.utils import to_categorical

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [ ]:
print(Y_train.shape)

(171, 8)


###特徵縮放

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print(X_train.shape)

(171, 9)


##TensorBoard

In [ ]:
%load_ext tensorboard

# Create TensorBoard log directory
import os
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard

logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

In [1]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


##模型建立

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras.losses import CategoricalCrossentropy


model = Sequential()


model.add(Dense(input_dim=X_train.shape[1], units=171, kernel_initializer="glorot_normal", activation="relu"))


model.add(Dense(units=38, kernel_initializer="glorot_normal", activation="relu"))


model.add(Dense(units=7, kernel_initializer="glorot_normal", activation="softmax"))


model.compile(optimizer="nadam", loss=CategoricalCrossentropy(), metrics=["acc"])

In [ ]:
#from HappyML.neural_networks import create_seq_model


#model = create_seq_model(nodes=[171, 64, 7])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 171)               1710      
                                                                 
 dense_1 (Dense)             (None, 38)                6536      
                                                                 
 dense_2 (Dense)             (None, 7)                 273       
                                                                 
Total params: 8519 (33.28 KB)
Trainable params: 8519 (33.28 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


##模型訓練與修正

In [ ]:
epochs_metrics = model.fit(x=X_train, y=Y_train, validation_split=0.2, batch_size=10, epochs=50, callbacks=[tensorboard_callback])

Epoch 1/50


ValueError: ignored

##超參數尋優

In [ ]:
Y_train

In [ ]:
Y_train.shape


In [ ]:
model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_data=(X_test, Y_test), callbacks=[tensorboard_callback])

In [ ]:
# Initializing Hyper-parameters
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from HappyML.neural_networks import create_seq_model

# Setup the HParams directory
hparams_dir = os.path.join(logdir, 'validation')

# Define Hyper-parameters
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([32, 64, 128]))
HP_WEIGHT_INIT = hp.HParam('kernel_initializer', hp.Discrete(["random_normal", "glorot_uniform", "glorot_normal"]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adadelta', 'rmsprop', 'adam', 'nadam']))

HPARAMS = [HP_NUM_UNITS, HP_WEIGHT_INIT, HP_OPTIMIZER]

# Set the Standard of Evaluation
METRIC_ACC = "acc"

#Creating & configuring a log file writer
with tf.summary.create_file_writer(hparams_dir).as_default():
    hp.hparams_config(
        hparams=HPARAMS,
        metrics=[hp.Metric(METRIC_ACC, display_name='ACC')]
    )

# Count the total run
total_run = 1
for p in HPARAMS:
  total_run *= len(p.domain.values)

curr_run = 1
for num_units in HP_NUM_UNITS.domain.values:
    for weight_init in HP_WEIGHT_INIT.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_WEIGHT_INIT: weight_init,
                HP_OPTIMIZER: optimizer
            }

            # Show somethings on Standard Output for progress
            run_name = "{} of {}".format(curr_run, total_run)
            print('--- Starting trial: {}'.format(run_name))
            print({h.name: hparams[h] for h in hparams})

            # Write information into log
            with tf.summary.create_file_writer(hparams_dir + "/" + run_name).as_default():
                hp.hparams(hparams)

                # Create & Train the model
                theNodes = [171, hparams[HP_NUM_UNITS], 8]
                model = create_seq_model(nodes=theNodes, weight_init=hparams[HP_WEIGHT_INIT], opt_name=hparams[HP_OPTIMIZER])
                model.fit(x=X_train, y=Y_train, validation_split=0.2, batch_size=10, epochs=5, callbacks=[tensorboard_callback, hp.KerasCallback(hparams_dir, hparams)])

                # Evaluate the Accuracy & Write to log
                test_loss, test_acc = model.evaluate(X_test, Y_test)
                tf.summary.scalar(METRIC_ACC, test_acc, step=10)
            curr_run += 1